This notebook mirrors the Korg basics notebook, but uses the JAX port (`korg_jax`) with the **local files in this repo**.

It uses:
- `basics/linelist.vald`
- `basics/s6000_g+1.0_m0.5_t05_st_z+0.00_a+0.00_c+0.00_n+0.00_o+0.00_r+0.00_s+0.00.mod`


In [1]:
lines = read_linelist(os.path.join(repo_root, "basics", "linelist.vald"), format="vald")
lines


NameError: name 'read_linelist' is not defined

## Read in a linelist (local file)


In [2]:
lines = read_linelist(os.path.join(repo_root, "basics", "linelist.vald"), format="vald")
lines


FileNotFoundError: [Errno 2] No such file or directory: 'basics/linelist.vald'

## Read a model atmosphere (local file)


In [ ]:
atm = read_model_atmosphere(os.path.join(
    repo_root,
    "basics",
    "s6000_g+1.0_m0.5_t05_st_z+0.00_a+0.00_c+0.00_n+0.00_o+0.00_r+0.00_s+0.00.mod"
))
atm


## Synthesizing a spectrum (timing)


In [ ]:
# run this cell again to see how much faster it is after precompilation.
A_X = format_A_X(0)

t = time.process_time()
sol_4000 = KJ.synthesize.synthesize(atm, lines, A_X, 4000, 4030)
print(time.process_time() - t)


In [ ]:
# run this cell again to see how much faster it is after precompilation.
t = time.process_time()
sol = KJ.synthesize.synthesize(atm, lines, A_X, 5000, 5030)
print(time.process_time() - t)


`synthesize` returns an object containing the spectrum and other useful quantities.


In [ ]:
sol.flux
# sol.wavelengths
# sol.cntm


We can plot them like this:


In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(sol.wavelengths, np.array(sol.flux) / np.array(sol.cntm), "k-")
plt.xlabel("$\lambda$ [Å]")
plt.ylabel("rectified flux");


## Specifying abundances


In [ ]:
metal_poor_A_X = format_A_X(-0.5)  # [M/H] = -1/2
alpha_rich_A_X = format_A_X(0, 0.5)  # all [M/H] = 0, but [alpha/H] = 0.5
Ni_enriched_A_X = format_A_X({"Ni": 1.0})  # all [M/H] = 0, except [Ni/H] = 1.0


In order to make sure that the model atmosphere abundances match the synthesis abundances,
we pass `A_X` vectors directly to the atmosphere generator. For the JAX port, this uses the **same
model atmosphere file** for now (no MARCS grid in repo), so we reuse the file-based atmosphere.


In [ ]:
metal_poor_sol = KJ.synthesize.synthesize(atm, lines, metal_poor_A_X, 5015, 5025)
alpha_rich_sol = KJ.synthesize.synthesize(atm, lines, alpha_rich_A_X, 5015, 5025)
Ni_enriched_sol = KJ.synthesize.synthesize(atm, lines, Ni_enriched_A_X, 5015, 5025)

plt.figure(figsize=(12, 4))
plt.plot(metal_poor_sol.wavelengths, metal_poor_sol.flux, label="metal-poor")
plt.plot(alpha_rich_sol.wavelengths, alpha_rich_sol.flux, label="alpha-rich")
plt.plot(Ni_enriched_sol.wavelengths, Ni_enriched_sol.flux, label="Ni enhanced")

plt.xlabel("$\lambda$ [Å]")
plt.ylabel("$F_\lambda/R_\mathrm{star}^2$ [erg s$^{-1}$ cm$^{-5}$]")
plt.xlim(5015, 5025)
plt.ylim(0, plt.ylim()[1] * 1.3)
plt.legend();


## Other data returned by `synthesize`


In [ ]:
plt.plot(sol.wavelengths, np.array(sol.alpha).transpose())
plt.yscale("log")
plt.xlabel("$\lambda$ [Å]")
plt.ylabel("α [cm$^{-1}$]");


In [ ]:
temps = atm.temp

for spec in ["H I", "H II", "O I", "OH"]:
    s = KJ.species.Species.from_string(spec)
    plt.plot(temps, sol.number_densities[s], label=spec)

plt.legend()
plt.yscale("log")
plt.xlabel("$T$ [K]")
plt.ylabel("$n$ [cm$^{-3}$]");
